# Verify C++ Optimizer Results

In [ ]:
import numpy as np
import pandas as pd
import math
from numba import njit

TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125], dtype=np.float64)
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5], dtype=np.float64)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def calc_total_score(df):
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    total = 0.0
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        total += score_group(xs, ys, ds, TX, TY)
    return total

# Warm up
_ = score_group(np.array([0.0]), np.array([0.0]), np.array([45.0]), TX, TY)

In [ ]:
# Calculate scores for all files\nimport os\nos.chdir('/home/code/experiments/003_cpp_optimizer')\n\nbaseline_df = pd.read_csv('baseline.csv')\noptimized_df = pd.read_csv('optimized.csv')\nbbox3_df = pd.read_csv('bbox3_output.csv')\n\nbaseline_score = calc_total_score(baseline_df)\noptimized_score = calc_total_score(optimized_df)\nbbox3_score = calc_total_score(bbox3_df)\n\nprint(f\"Baseline score: {baseline_score:.6f}\")\nprint(f\"tree_packer optimized score: {optimized_score:.6f}\")\nprint(f\"bbox3 optimized score: {bbox3_score:.6f}\")\nprint(f\"\\nImprovement from tree_packer: {baseline_score - optimized_score:.6f}\")\nprint(f\"Improvement from bbox3: {baseline_score - bbox3_score:.6f}\")